In [ ]:
### RankNet-check-modeldata

Check the intermediate data in the network.


In [5]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import mxnet as mx
from mxnet import gluon
import pickle
import json
import random
import inspect
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import indycar.model.stint_simulator_shortterm_pitmodel as stint

INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU
INFO:root:Using GPU


In [9]:
### load test_ds and model saved data
prediction_length = 2
freq = "1T"
cardinality = [0]

def load_dataset(inputfile):
    global freq, prediction_length, cardinality

    with open(inputfile, 'rb') as f:
        # have to specify it.
        freq, prediction_length, cardinality,train_ds, test_ds = pickle.load(f, encoding='latin1')
    
    print(f"number of cars: {cardinality}")
    
    return train_ds, test_ds

def load_modeldata(inputfile):
    with open(inputfile, 'rb') as f:
        # have to specify it.
        savedata = pickle.load(f, encoding='latin1')
    
    print(f"columns of data: {savedata.keys()}")
    
    return savedata


In [49]:
testfile = 'testset-car12-f1min-t2-rIndy500-2018-gluonts-indy-2018.pickle'
#100 samples
#modelfile = 'ranknet_savedata_multilayers.pickle'
#1 sample
modelfile = 'ranknet_savedata_multilayers_s1.pickle'


In [50]:
_, test_ds = load_dataset(testfile)
savedata = load_modeldata(modelfile)

number of cars: [58]
columns of data: dict_keys(['input', 'target', 'lags', 'theta', 'hstate', 'rnnoutput'])


In [21]:
### iter through the test set
testset = list(test_ds)
print('testset predict laps start:', testset[-1]['target'].shape,
     'end:', testset[0]['target'].shape
     )

testset predict laps start: (13,) end: (200,)


In [25]:
# lap id start from 0
pitstops = np.where(testset[0]['feat_dynamic_real'][1]==1)
pitstops

(array([ 31,  49,  93, 128, 170]),)

In [59]:
len(savedata['target'])

188

In [51]:
savedata['target'][0].shape

(100, 1)

In [60]:
savedata['target'][-1].shape

(100, 1)

In [64]:
savedata['target'][0].reshape(-1)

array([-0.44145086,  1.5676428 ,  1.1341344 ,  0.71432924,  1.1557796 ,
        0.6971927 ,  1.4773084 ,  1.310024  ,  5.6919284 , -0.44081745,
        0.6274962 ,  1.4137928 ,  0.8880926 ,  1.5269084 ,  1.9035591 ,
       -0.09946428, -7.9242    ,  0.9420268 ,  0.9460869 ,  1.1100156 ,
        0.5714886 ,  1.2271032 ,  1.8293935 , -0.26692703,  1.1405493 ,
        1.347515  ,  1.4340081 ,  0.9209535 ,  0.8217745 ,  1.9705166 ,
        0.84749085,  0.9946999 ,  0.3822164 ,  0.246796  ,  0.24754158,
        1.0472313 ,  1.2376559 ,  1.273707  ,  1.8893508 ,  0.5011305 ,
        0.98692626,  1.9640998 , -1.7275789 ,  1.3330501 ,  0.6332364 ,
        1.285609  ,  0.65965104,  2.56647   ,  1.816179  ,  1.2097907 ,
       -1.1008717 ,  2.2113101 ,  0.35411757, -6.926241  ,  1.2208822 ,
        1.5080682 ,  0.9842256 ,  0.76948315,  1.1269274 ,  1.1351879 ,
        1.0171196 ,  0.9659873 ,  0.32233703,  0.4349263 ,  1.6476487 ,
        1.9735689 ,  1.5934758 ,  1.7168025 ,  0.62858397,  2.14

In [65]:
savedata['target'][-1].reshape(-1)

array([3.0077455, 2.9992945, 3.0057023, 2.9898915, 2.9992082, 3.0373669,
       3.0118728, 2.9974728, 2.9922664, 3.0066533, 3.005501 , 3.0021725,
       3.0109591, 3.0040984, 3.0032787, 3.0034544, 3.0118995, 3.0025365,
       3.0018969, 3.002289 , 3.021418 , 3.0082383, 3.0040195, 3.0018585,
       3.0087698, 3.0286522, 2.9901762, 3.005068 , 3.0017397, 3.0210695,
       2.998493 , 3.0039122, 3.0300102, 2.988176 , 3.0007455, 2.9993865,
       3.0009956, 3.0072436, 2.9894586, 3.0032027, 3.0080206, 2.9985619,
       2.9962795, 3.0310862, 3.0048695, 3.0011578, 2.9882932, 2.9976094,
       2.9952745, 3.00089  , 3.000582 , 2.994224 , 2.9979656, 2.9967551,
       3.006228 , 3.0035195, 2.9964004, 3.0000408, 3.002852 , 3.0074053,
       3.0070722, 3.0062852, 3.0023658, 3.0089364, 3.0021088, 3.0066552,
       3.0000393, 3.0121229, 2.9886208, 2.9966712, 3.007021 , 3.0048664,
       3.0008152, 3.0221179, 2.9953732, 3.0097713, 2.9968925, 2.9930484,
       3.0039742, 3.0031986, 2.998498 , 3.0035753, 

In [52]:
len(savedata['input'])

188

In [53]:
savedata['input'][0].shape

(1, 40, 33)

In [54]:
len(savedata['theta'][0])

3

In [66]:
savedata['theta'][0][0].reshape(-1)


[ 0.2113398   0.31259346  0.30035985  0.25674474  0.24849926  0.18456659
  0.2923451   0.21115455  0.32288575  0.18292964 -0.02808346  0.3482263
  0.19701114  0.39323848  0.38089508  0.03942234 -1.7029319   0.23678763
  0.24280587  0.22648484  0.11472401  0.21699321  0.2489924   0.0190714
  0.33308488  0.30935138  0.3021624   0.18568337  0.1694366   0.43320858
  0.18486372  0.28720713  0.2278513   0.20472297  0.09632437  0.22742592
  0.18488745  0.2546913   0.43022138  0.0951069   0.31564718  0.33118922
  0.01522211  0.282713    0.1202973   0.24679287  0.17332536  0.22852504
  0.31322873  0.28692096  0.37347323  0.27579212  0.25569117  0.10909198
  0.3192501   0.3381303   0.20826252  0.25301716  0.30314913  0.2774055
  0.33302093  0.3432911   0.21460994  0.2124842   0.2674272   0.4151464
  0.32267636  0.4036957   0.21642789  0.3404966   0.11972626  0.31998557
  0.2838149   0.36183983  0.27487925  0.3583609   0.16023585  0.26283923
  0.24284525  0.16814554  0.35996652  0.08448908  0.34

In [67]:
savedata['theta'][-1][0].reshape(-1)


[1.066111  1.0637047 1.0665786 1.0632758 1.064501  1.0665865 1.0656804
 1.0636574 1.0619617 1.0673927 1.0657667 1.0655301 1.0686353 1.064816
 1.0666254 1.0656897 1.0670568 1.0651155 1.0652413 1.0651318 1.0730463
 1.0670315 1.0645143 1.0652158 1.0664655 1.0663153 1.0622551 1.0658764
 1.0664176 1.0708448 1.0656103 1.0659695 1.0752114 1.0654832 1.0651457
 1.064361  1.0644644 1.0658069 1.06027   1.0652804 1.0649706 1.0640204
 1.0657115 1.0759063 1.0661377 1.064275  1.0618122 1.0652609 1.0642047
 1.0664177 1.065689  1.062489  1.0626836 1.0651668 1.0669981 1.065185
 1.0627176 1.0649273 1.064258  1.065026  1.0680534 1.0664967 1.0645912
 1.0685518 1.0659287 1.0657933 1.0649462 1.0661708 1.065636  1.0633123
 1.0668445 1.0649129 1.0639756 1.0727793 1.065102  1.0667129 1.0662006
 1.063021  1.0657257 1.0664953 1.0648925 1.0645307 1.0666456 1.0638584
 1.0622164 1.0644053 1.0655134 1.0620639 1.065827  1.0648849 1.0644757
 1.0648177 1.0678092 1.0656393 1.0642436 1.0635877 1.0650789 1.0649594
 1.0652

In [68]:
savedata['rnnoutput'][0].shape

(100, 1, 40)

In [69]:
rnn = savedata['rnnoutput'][0]
rnn[0]

array([[ 0.37186566, -0.23556681,  0.03524104,  0.31859934,  0.10533482,
         0.01034962,  0.15312192, -0.06566573, -0.17070529,  0.06987295,
        -0.2244168 ,  0.03112466, -0.04584079, -0.23805699, -0.09133454,
        -0.08574519, -0.06131275, -0.03852127,  0.21195891, -0.29207027,
         0.0055407 ,  0.04884779, -0.32507932,  0.00214675, -0.04041872,
        -0.22301807, -0.09570426,  0.02288406, -0.07174078, -0.221968  ,
         0.08084088, -0.33736727,  0.33693063, -0.2073015 ,  0.44099346,
         0.30486134,  0.13770637, -0.06217545, -0.07174945, -0.28200722]],
      dtype=float32)

In [70]:
rnn[1]

array([[ 0.37727368, -0.24469908,  0.01767111,  0.3255163 ,  0.11553065,
         0.01072059,  0.13672808, -0.07457735, -0.19335678,  0.06119858,
        -0.23116684,  0.03275276, -0.05076409, -0.26916248, -0.06966979,
        -0.11781626, -0.05174231, -0.0365288 ,  0.20475395, -0.27779245,
         0.00456455,  0.04795628, -0.30416936, -0.02512237, -0.01391805,
        -0.1918397 , -0.10072944,  0.02336511, -0.05365315, -0.23151982,
         0.10357203, -0.33330145,  0.33445174, -0.20318173,  0.4490133 ,
         0.3366268 ,  0.15412374, -0.06943412, -0.07026852, -0.2688479 ]],
      dtype=float32)

In [71]:
rnn[32]

array([[ 0.37274674, -0.23706463,  0.03235728,  0.31974936,  0.10702126,
         0.01040592,  0.15047304, -0.06710908, -0.17449358,  0.06845737,
        -0.2255145 ,  0.03139009, -0.0466314 , -0.2432203 , -0.08781262,
        -0.09099396, -0.05976961, -0.03820486,  0.21077666, -0.28974897,
         0.00540547,  0.04869879, -0.32164904, -0.00231871, -0.03613469,
        -0.21805382, -0.09652708,  0.0228952 , -0.06877895, -0.22353329,
         0.08449088, -0.3366885 ,  0.33652785, -0.20664468,  0.44231206,
         0.3100715 ,  0.14037058, -0.06329674, -0.07152263, -0.2799398 ]],
      dtype=float32)

In [72]:
len(savedata['hstate'][0])

4

In [73]:
savedata['hstate'][0][0].shape

(100, 40)